In [1]:
import os
import pandas as pd
import numpy as np
import h5py
import pickle
from tflearn.data_utils import pad_sequences
import tensorflow as tf
from numba import jit
import codecs

/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhangqifan/anaconda3/envs

Instructions for updating:
non-resource variables are not supported in the long term


/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhan

### 数据预处理封装函数

In [15]:
def dislodge_stopwords(words_list):
    '''
    去停用词
    '''
    res = []
    stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
    for i in words_list:
        if i not in stopwords:
            res.append(i)
        
    return res


def transform_label_as_onehot(label_list,label_size=19):
    '''
    标签制作onehot
    '''
    result=np.zeros(label_size)
    result[label_list] = 1
    
    return result


def get_X_Y(train_data_x,train_data_y,label_size,word2index):
    '''
    获取训练数据集和标签
    '''
    X=[]
    Y=[]

    for index, row in train_data_x.iterrows():
        text_words=row['text_words']
        #转换成id
        text_word_id_list=[word2index.get(x,value) for x in text_words if x.strip()]
        
        X.append(text_word_id_list)

    for index, row in train_data_y.iterrows():
        #label是从1开始的，索引从0开始的
        label_ids=row['label']-1
        label_list_dense=[label_ids]
        label_list_sparse=transform_label_as_onehot(label_list_dense,label_size)
        Y.append(label_list_sparse)

    return X,Y


def save_data(cache_file_h5py,cache_file_pickle,word2index,label2index,train_X,train_Y,valid_X,valid_Y,test_X,test_Y):
    '''
    保存训练数据和词表标签列表
    '''
    f = h5py.File(cache_file_h5py, 'w')
    f['train_X'] = train_X
    f['train_Y'] = train_Y
    f['valid_X'] = valid_X
    f['valid_Y'] = valid_Y
    f['test_X'] = test_X
    f['test_Y'] = test_Y
    f.close()

    with open(cache_file_pickle, 'ab') as target_file:
        pickle.dump((word2index,label2index), target_file)

### 预处理：生成模型训练所需文件

In [16]:
#读取已分割好的训练集和测试集
df_train = pd.read_csv('data/train_ngram_tfidf_data.txt',header=None,sep='\t',names=['label','text'],encoding='GB18030')
df_test = pd.read_csv('data/test_ngram_tfidf_data.txt',header=None,sep='\t',names=['label','text'],encoding='GB18030')

In [17]:
#制作正文的词表与长度(word_seg)
df_train['text_words'] = df_train['text'].map(lambda x:x.split(' '))
df_train['text_length'] = df_train['text_words'].map(lambda x:len(x))
df_test['text_words'] = df_test['text'].map(lambda x:x.split(' '))
df_test['text_length'] = df_test['text_words'].map(lambda x:len(x))
#训练集测试集合并
df_all = pd.concat([df_train,df_test])

In [18]:
df_train

label                                               text  \
0         15  ivvi智能 手环 智能 智能手环 ivvi ivvi智能手环 来袭 暖心 助手暖心来袭 运...   
1         13  眼镜 ar ar眼镜 爱普生 官方 旗舰店 官方旗舰店 眼镜 ar ar眼镜 正品 正品底价...   
2          9  交通显示屏 led 云南 显示屏 led交通 云南led 云南led交通 交通 公司 交通显...   
3         13  眼镜 ar ar眼镜 爱普生 bt 300 眼镜 ar ar眼镜 助力 指导 助力远程指导 ...   
4          7  ps5发售日期 发售日期 ps5 价格 硬件配置 新闻 汇总 新闻汇总 ps5新闻 ps5新...   
...      ...                                                ...   
15151     17  电话 维修售后 自动关机维修 售后 电视自动关机 维修维修 自动关机 维修售后电话 维修 自...   
15152      2  易购手机 信易购手机 易购 信易购 手机 办理分期 办理 分期 专业 分期 专业手机 手机 ...   
15153      8  云主机 主机 华为 华为云 来袭 特惠 特惠来袭 623 核4g服务器 2核 623元 核4...   
15154      9  间距 led 河南小 河南小间距 间距led显示屏 河南 小间距led 显示屏 厂家 小间距...   
15155     12  全网 标牌 数字 数字标牌 30秒快速报价 30秒 快速报价 报价 快速 30秒快速 热销 ...   

                                              text_words  text_length  
0      [ivvi智能, 手环, 智能, 智能手环, ivvi, ivvi智能手环, 来袭, 暖心,...           68  
1      [眼镜, ar, ar眼镜, 爱普生, 官方, 旗舰店, 官方旗舰店, 眼镜, ar, ar...          121  
2      [交通显示屏, led, 云南, 显示屏, led交通, 云南led, 云南led交通, 交...          223  
3      [眼镜, ar, ar眼镜, 爱普生, bt, 300, 眼镜, ar, ar眼镜, 助力,...          126  
4      [ps5发售日期, 发售日期, ps5, 价格, 硬件配置, 新闻, 汇总, 新闻汇总, p...           69  
...                                                  ...          ...  
15151  [电话, 维修售后, 自动关机维修, 售后, 电视自动关机, 维修维修, 自动关机, 维修售...           96  
15152  [易购手机, 信易购手机, 易购, 信易购, 手机, 办理分期, 办理, 分期, 专业, 分...          106  
15153  [云主机, 主机, 华为, 华为云, 来袭, 特惠, 特惠来袭, 623, 核4g服务器, ...          141  
15154  [间距, led, 河南小, 河南小间距, 间距led显示屏, 河南, 小间距led, 显示...          214  
15155  [全网, 标牌, 数字, 数字标牌, 30秒快速报价, 30秒, 快速报价, 报价, 快速,...          145  

[15156 rows x 4 columns]

In [19]:
df_all.describe()

label   text_length
count  19000.00000  19000.000000
mean      10.00000    141.281421
std        5.47737     59.435180
min        1.00000      5.000000
25%        5.00000     98.000000
50%       10.00000    144.000000
75%       15.00000    182.000000
max       19.00000    341.000000

In [20]:
#求85%分位数，确定max_length
df_all.text_length.sort_values().iloc[int(df_all.shape[0]*0.85)]
#pd.concat([df_train.text_length,df_test.text_length]).sort_values().iloc[16150]

203

In [21]:
#制作全量词表并保存
word2index={'PAD':0,'UNK':1,'CLS':2,'SEP':3,'unused1':4,'unused2':5,'unused3':6,'unused4':7,'unused5':8}
word_count = 9 #word2index长度
vocab_word_object = open('data/vocab.txt','w')
for key,value in word2index.items():
    vocab_word_object.write(key+'\n')
for word_lst in df_all['text_words']:
    for word in word_lst:
        if word not in word2index:
            word2index[word] = word_count
            word_count += 1
            vocab_word_object.write(word+'\n')
vocab_word_object.close()

In [22]:
#制作标签列表并保存
label2index = \
{
'IT维修':17,
'IT回收':16,
'显示器':9,
'服务器':8,
'摄影器材':4,
'电脑':1,
'手机':2,
'IT租赁':19,
'耳机售卖':5,
'音响':11,
'车载产品':6,
'监控设备':10,
'打印机':14,
'电视':3,
'终端机':12,
'游戏机':7,
'恢复/删除数据':18,
'无人机':13,
'手环':15
}
label_target_object = open('data/label_set.txt','w')
for key,value in label2index.items():
    label_target_object.write(str(value)+'\n')
label_target_object.close()

In [24]:
#设置参数
label_size = len(label2index)
cache_path_h5py = 'data/data.h5'
cache_path_pickle = 'data/vocab_label.pik'
max_sentence_length = 200 #203去掉零头了


#获取XY集合 (这里验证集和测试集是一样的)
num_examples=df_all.shape[0]
df_all_train = df_train
df_all_valid = df_test
df_all_test = df_test
train_X,train_Y=get_X_Y(df_all_train,df_all_train,label_size,word2index)
valid_X,valid_Y=get_X_Y(df_all_valid,df_all_valid,label_size,word2index)
test_X,test_Y=get_X_Y(df_all_test,df_all_test,label_size,word2index)


#裁剪和截断
train_X = pad_sequences(train_X, maxlen=max_sentence_length, value=0.)
valid_X = pad_sequences(valid_X, maxlen=max_sentence_length, value=0.)
test_X = pad_sequences(test_X, maxlen=max_sentence_length, value=0.)

#转成array
train_X = np.array(train_X); train_Y=np.array(train_Y)
valid_X = np.array(valid_X); valid_Y=np.array(valid_Y)
test_X = np.array(test_X); test_Y=np.array(test_Y)

print("全部数据量:",num_examples)
print("train_X:",train_X.shape,";train_Y:",train_Y.shape,";valid_X.shape:",valid_X.shape,";valid_Y:",valid_Y.shape,";test_X:",test_X.shape,";test_Y:",test_Y.shape)

#保存文件
save_data(cache_path_h5py,cache_path_pickle,word2index,label2index,train_X,train_Y,valid_X,valid_Y,test_X,test_Y)


全部数据量: 19000
train_X: (15156, 200) ;train_Y: (15156, 19) ;valid_X.shape: (3844, 200) ;valid_Y: (3844, 19) ;test_X: (3844, 200) ;test_Y: (3844, 19)


### 模型训练

In [159]:
#执行训练，并打印训练log
!python TextCNN_train.py

/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhangqifan/anaconda3/envs

use single layer CNN
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0118 15:47:26.129475 139672302290688 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0118 15:47:26.283312 139672302290688 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f07d0afbcd0>> co


W0118 15:47:26.949454 139672302290688 deprecation_wrapper.py:119] From TextCNN_train.py:70: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

初始化变量
2021-01-18 15:47:27.155054: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
trainX[start:end]: [[   9   10   11 ...    0    0    0]
 [  36   37   38 ...    0    0    0]
 [  80   81   82 ...    0    0    0]
 ...
 [ 759  168  759 ...    0    0    0]
 [ 919 1365 1366 ...    0    0    0]
 [1371 1372 1373 ...    0    0    0]]
OMP: Info #252: KMP_AFFINITY: pid 22982 tid 23013 thread 1 bound to OS proc set 2
OMP: Info #252: 

Epoch 5	Batch 50	Train Loss:0.613	Learning rate:0.00030
Epoch 5	Batch 100	Train Loss:0.608	Learning rate:0.00030
Epoch 5	Batch 150	Train Loss:0.602	Learning rate:0.00030
Epoch 5	Batch 200	Train Loss:0.596	Learning rate:0.00030
epoch增加一轮
5 1 True
P: 99.66%, R: 99.67%, Micro_f1: 99.66%, Macro_f1: 99.66%, Accuracy: 99.66
Epoch 5 Validation Loss:0.609	F1 Score:0.997	F1_micro:0.997	F1_macro:0.997
P: 99.66%, R: 99.67%, Micro_f1: 99.66%, Macro_f1: 99.66%, Accuracy: 99.66
Epoch 6 Validation Loss:0.608	F1 Score:0.997	F1_micro:0.997	F1_macro:0.997
开始保存模型
Epoch 6	Batch 50	Train Loss:0.560	Learning rate:0.00030
Epoch 6	Batch 100	Train Loss:0.552	Learning rate:0.00030
Epoch 6	Batch 150	Train Loss:0.547	Learning rate:0.00030
Epoch 6	Batch 200	Train Loss:0.542	Learning rate:0.00030
epoch增加一轮
6 1 True
P: 99.71%, R: 99.72%, Micro_f1: 99.71%, Macro_f1: 99.71%, Accuracy: 99.71
Epoch 6 Validation Loss:0.552	F1 Score:0.997	F1_micro:0.997	F1_macro:0.997
P: 99.71%, R: 99.72%, Micro_f1: 99.71%, Macro_f1: 99.7

### 预测

In [13]:
#待预测样本格式如下
df_test

label                                               text  \
0        14  联想打印机 打印机 联想打印机年终 打印机年终 联想 年终盛惠 打印机年终盛惠 年终 盛惠 ...   
1         5  全新 airpods pods air pro 抗汗 汗抗 抗汗抗 抗水 汗抗水 不同凡响 ...   
2        16  南昌 回收 南昌手机回收 南昌手机 手机 手机回收 二手手机 二手手机 赣鸿 免费 免费估价...   
3         5  powerbeats beats power pro 无线高性能 高性能耳机全面上市 耳机 ...   
4        14  madica madica证卡打印机 madica证卡打印机价格 证卡打印机价格 证卡打印机...   
...     ...                                                ...   
3839      6  康凯斯 定位器生产厂家 定位器 生产厂家 车辆定位器 车辆 车辆定位器生产厂家 专业 定位器...   
3840     13  无人机哪家 哪家 无人机哪家好 哪家好 无人机 稳定 稳定4k画质 超清 清稳定 清稳定4k...   
3841      9  找深圳 led地铁屏 led 厂家找深圳 百发 屏厂家找 led地铁 找深圳百发 厂家找 深...   
3842     19  电脑租赁 立租 押金电脑租赁 免押金 一站式免 一站式 押金 一站式免押金 免押金电脑 电脑...   
3843      8  腾讯云 腾讯 服务器 云服务器 1核 1核2g 2g 核2g 99 99元 1年 高安全 高...   

                                             text_words  text_length  
0     [联想打印机, 打印机, 联想打印机年终, 打印机年终, 联想, 年终盛惠, 打印机年终盛惠...           47  
1     [全新, airpods, pods, air, pro, 抗汗, 汗抗, 抗汗抗, 抗水,...           15  
2     [南昌, 回收, 南昌手机回收, 南昌手机, 手机, 手机回收, 二手手机, 二手手机, 赣...           20  
3     [powerbeats, beats, power, pro, 无线高性能, 高性能耳机全面...          104  
4     [madica, madica证卡打印机, madica证卡打印机价格, 证卡打印机价格, ...          117  
...                                                 ...          ...  
3839  [康凯斯, 定位器生产厂家, 定位器, 生产厂家, 车辆定位器, 车辆, 车辆定位器生产厂家...           66  
3840  [无人机哪家, 哪家, 无人机哪家好, 哪家好, 无人机, 稳定, 稳定4k画质, 超清, ...           34  
3841  [找深圳, led地铁屏, led, 厂家找深圳, 百发, 屏厂家找, led地铁, 找深圳...          188  
3842  [电脑租赁, 立租, 押金电脑租赁, 免押金, 一站式免, 一站式, 押金, 一站式免押金,...          207  
3843  [腾讯云, 腾讯, 服务器, 云服务器, 1核, 1核2g, 2g, 核2g, 99, 99...          223  

[3844 rows x 4 columns]

In [161]:
#将待预测文本输出到待预测文件夹（注意这里输出的是默认utf-8，如需‘GB8030’增加encoding=‘GB8030’，并在TextCNN_predict.py中更改文件读取方式）
df_test.to_csv('need_to_predict/test.csv')

In [19]:
#执行预测，预测结果写入predict_result文件夹下
!python TextCNN_predict.py

/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhangqifan/anaconda3/envs

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0122 12:00:59.560659 140273402562304 deprecation.py:506] From /home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0122 12:00:59.982583 140273402562304 lazy_loader.py:50] 
The TensorFlow contrib module will


W0122 12:01:00.236215 140273402562304 deprecation_wrapper.py:119] From /home/data/zhangqifan/data/def-research/projects/IT_consume_electronics/TextCNN_model.py:134: The name tf.train.exponential_decay is deprecated. Please use tf.compat.v1.train.exponential_decay instead.


W0122 12:01:00.240496 140273402562304 deprecation_wrapper.py:119] From /home/data/zhangqifan/data/def-research/projects/IT_consume_electronics/TextCNN_model.py:136: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.


W0122 12:01:00.683068 140273402562304 deprecation_wrapper.py:119] From TextCNN_predict.py:54: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

Restoring Variables from Checkpoint
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0122 12:01:00.731493 140273402562304 deprecation.py:323] From /home/zhangqifan/anaconda3/envs/textcnn/lib/python3.7/site-packages/tensorflow/python/traini

### 效果验证和准召

In [20]:
#读取预测结果文件
df_pre_res = pd.read_csv('predict_result/test_res.csv',header=None,sep='\t')

In [21]:
#提取出预测结果并映射成id
df_pre_res['predict'] = df_pre_res[1].map(lambda x:label2index[x.split(',')[0]])
#标签和预测结果合并
df_label_predict = pd.concat([df_test['label'].reset_index(drop=True),df_pre_res['predict'].reset_index(drop=True)],axis=1)

In [22]:
#查看真实标签和预测结果
df_label_predict

label  predict
0        14       14
1         5        5
2        16       16
3         5        5
4        14       14
...     ...      ...
3839      6        6
3840     13       13
3841      9        9
3842     19       19
3843      8        8

[3844 rows x 2 columns]

In [166]:
#查看各个label预测的数量
df_label_predict.predict.value_counts(dropna=False)

9     225
3     222
14    214
19    209
15    208
10    207
12    203
1     203
11    202
4     202
18    201
16    200
13    199
5     197
8     195
7     193
2     193
6     188
17    183
Name: predict, dtype: int64

In [167]:
#各标签的准召
for i in range(1,20):
    try:
        print('类别%s准确率:%s' % (str(i),str(round(df_label_predict.loc[(df_label_predict['predict']==i) & (df_label_predict['label']==i)].shape[0]/(df_label_predict.loc[df_label_predict['predict']==i].shape[0]),2))))                   
        print('类别%s召回率:%s' % (str(i),str(round(df_label_predict.loc[(df_label_predict['predict']==i) & (df_label_predict['label']==i)].shape[0]/(df_label_predict.loc[df_label_predict['label']==i].shape[0]),2))))                   
        print('---------------')
    except:
        print('类别%s无数据' % (str(i)))
        print('---------------')

类别1准确率:0.99
类别1召回率:1.0
---------------
类别2准确率:0.99
类别2召回率:1.0
---------------
类别3准确率:1.0
类别3召回率:1.0
---------------
类别4准确率:0.99
类别4召回率:1.0
---------------
类别5准确率:1.0
类别5召回率:0.99
---------------
类别6准确率:1.0
类别6召回率:1.0
---------------
类别7准确率:1.0
类别7召回率:0.99
---------------
类别8准确率:0.99
类别8召回率:1.0
---------------
类别9准确率:0.99
类别9召回率:1.0
---------------
类别10准确率:1.0
类别10召回率:1.0
---------------
类别11准确率:1.0
类别11召回率:1.0
---------------
类别12准确率:1.0
类别12召回率:0.99
---------------
类别13准确率:1.0
类别13召回率:1.0
---------------
类别14准确率:1.0
类别14召回率:1.0
---------------
类别15准确率:1.0
类别15召回率:1.0
---------------
类别16准确率:1.0
类别16召回率:1.0
---------------
类别17准确率:1.0
类别17召回率:0.99
---------------
类别18准确率:1.0
类别18召回率:1.0
---------------
类别19准确率:1.0
类别19召回率:1.0
---------------
